In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os


In [2]:

if not os.path.exists('./gen_img'):
    os.mkdir('./gen_img')



In [23]:

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 252, 428)
    return x


num_epochs = 100
batch_size = 1
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.Resize((256,420)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



In [24]:
# img_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def load_dataset():
    data_path = './gen_img/'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform= img_transform
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True
    )
    return train_loader

# for batch_idx, (data, target) in enumerate(load_dataset()):
    
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [25]:
dataloader = load_dataset()

In [26]:
from PIL import Image

In [33]:


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        
        
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2),  # b, 8, 2, 2
            nn.Conv2d(8, 4, 3, stride=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2)  # b, 8, 2, 2
        )
        
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4, 8, 4, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 16, 3, stride=2, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 3, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        print(x.shape)
        x = self.decoder(x)
        print(x.shape)
        return x




In [34]:
num_epochs = 4000

In [35]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data.cpu()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './gen_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder.pth')

RuntimeError: CUDA out of memory. Tried to allocate 58.38 MiB (GPU 0; 5.77 GiB total capacity; 4.63 GiB already allocated; 32.69 MiB free; 13.62 MiB cached)